# Getting Started with SUBER

SUBER is complex so we will take it one module at a time. The entry point for suber is [algorithms.mind.CFTrain2](https://github.com/acsheller/SUBER/blob/main/algorithms/mind/CF_train_A2C2.py).  One can review this file and simply get started. 

This is the baseline model that shows SUBER working for 

In [1]:
import os
import sys
import numpy as np
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from algorithms.wrappers import StableBaselineWrapperNum
from environment.mind.configs import get_enviroment_from_args, get_base_parser
from environment import load_LLM
from algorithms.logging_config import get_logger

from stable_baselines3.common.callbacks import BaseCallback, CallbackList
from stable_baselines3.common.distributions import CategoricalDistribution
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3 import A2C
from stable_baselines3.common.preprocessing import get_flattened_obs_dim
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.type_aliases import TensorDict

import torch
import torch.nn as nn
import gymnasium as gym
from gymnasium import spaces
from typing import Callable, Tuple
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor

import argparse
from tqdm import tqdm, trange
logger = get_logger("suber_logger")

# Set environment variable to avoid tokenizer parallelism issues
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/app/sb3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

## This is an example call to the system.  

python3 -m algorithms.mind.CF_train_A2C2 --llm_model=TheBloke/Llama-2-13B-chat-GPTQ --llm_rater=2Shot_system_our --perturbator=gaussian --items_retrieval=most_similar_3_title --reward_shaping=exp_decay_time --embedding-dim=512 --gamma=0.95 --seed=42 --news_dataset=mind_dataset



In [2]:
def parse_args():
    parser = get_base_parser()
    parser.add_argument("--model_device", type=str, default="cuda:0")
    parser.add_argument("--gamma", type=float, default=0.975)
    parser.add_argument("--embedding_dim", type=int, default=32)
    parser.add_argument("--learning_rate", type=float, default=0.01)
    parser

    parser.add_argument(
        '--news_dataset', 
        choices=['mind_dataset', 'small_mind_dataset'], 
        help='Specify the news dataset to use',
        default='small_mind_dataset'
    )
    # TODO Parser arguments should be here -- need to consider as most of them are 
    # in config.py

    args, unknown = parser.parse_known_args()
    return args

In [3]:
# Linear schedule function
def linear_schedule(initial_value: float):
    def func(progress_remaining: float) -> float:
        current_value = initial_value * progress_remaining
        #print(f"Linear schedule called: progress_remaining={progress_remaining}, learning_rate={current_value}")
        #logger.info(f"Linear schedule called: progress_remaining={progress_remaining}, learning_rate={current_value}")
        return current_value
    return func

class CombinedCallback(BaseCallback):
    def __init__(self, save_freq=50000, log_freq=5000, save_path="./tmp/models/", name_prefix="rl_model", verbose=0):
        super(CombinedCallback, self).__init__(verbose)
        self.save_freq = save_freq
        self.log_freq = log_freq
        self.save_path = save_path
        self.name_prefix = name_prefix
        self.metrics = []

    def _on_step(self) -> bool:
        if self.n_calls % self.log_freq == 0:
            rewards = self.locals['rewards']
            episode_length = self.locals.get('episode_lengths', None)
            value_loss = self.locals.get('value_loss', None)
            policy_loss = self.locals.get('policy_loss', None)
            if value_loss is not None:
                log_message = {
                    "step": self.num_timesteps,
                    "reward": rewards,
                    "episode_length": episode_length,
                    "value_loss": value_loss,
                    "policy_loss": policy_loss
                }
                self.metrics.append(log_message)
                self.logger.info(log_message)


        if self.n_calls % self.save_freq == 0:
            model_path = f"{self.save_path}/{self.name_prefix}_{self.num_timesteps}_steps"
            self.model.save(model_path)
            if self.verbose > 0:
                print(f"Saving model checkpoint to {model_path}")
        return True


In [4]:
class Net(nn.Module):
    def __init__(self, obs_space: gym.spaces.Space, num_users: int, num_items: int, learning_rate: float = 0.001):
        super().__init__()
        embedding_dim = args.embedding_dim
        self.latent_dim_pi = embedding_dim * 2
        self.latent_dim_vf = embedding_dim * 2

        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.user_bias = nn.Embedding(num_users, 1)

        self.policy_net = nn.Sequential(
            nn.Linear(self.user_embedding.embedding_dim, embedding_dim * 2),
            nn.ReLU(),
            nn.Linear(embedding_dim * 2, embedding_dim * 4),
            nn.ReLU(),
            nn.Linear(embedding_dim * 4, num_items)
        )

        self.value_net = nn.Sequential(
            nn.Linear(self.user_embedding.embedding_dim + num_items, self.latent_dim_vf * 2),
            nn.ReLU(),
            nn.Linear(self.latent_dim_vf * 2, embedding_dim * 4),
            nn.ReLU(),
            nn.Linear(embedding_dim * 4, self.latent_dim_vf),
            nn.ReLU()
        )

    def forward(self, features: TensorDict) -> Tuple[torch.Tensor, torch.Tensor]:
        user_id = features["user_id"].squeeze(1)
        news_seen = features["items_interact"]

        user_embedding = self.user_embedding(user_id)
        user_embedding_value = torch.cat([user_embedding, news_seen], dim=1)
        user_bias = self.user_bias(user_id)

        mask = features["items_interact"].to(dtype=torch.bool)
        logits = self.policy_net(user_embedding) + user_bias
        logits[mask] = -torch.inf
        return logits, self.value_net(user_embedding_value)

    def forward_actor(self, features: TensorDict) -> torch.Tensor:
        user_id = features["user_id"].squeeze(1)
        user_embedding = self.user_embedding(user_id)
        user_bias = self.user_bias(user_id)

        mask = features["items_interact"].to(dtype=torch.bool)
        logits = self.policy_net(user_embedding) + user_bias
        logits[mask] = -torch.inf
        return logits

    def forward_critic(self, features: TensorDict) -> torch.Tensor:
        user_id = features["user_id"].squeeze(1)
        news_seen = features["items_interact"]

        user_embedding = self.user_embedding(user_id)
        user_embedding_value = torch.cat([user_embedding, news_seen], dim=1)
        return self.value_net(user_embedding_value)

class DistributionUseLogitsDirectly(CategoricalDistribution):
    def __init__(self, action_dim: int):
        super().__init__(action_dim)

    def proba_distribution_net(self, latent_dim: int) -> nn.Module:
        return nn.Identity(latent_dim)

class CustomActorCriticPolicy(ActorCriticPolicy):
    def __init__(self, observation_space: spaces.Space, action_space: spaces.Space, lr_schedule: Callable[[float], float], *args, **kwargs):
        kwargs["ortho_init"] = True
        super().__init__(observation_space, action_space, lr_schedule, *args, **kwargs)

        self.action_dist = DistributionUseLogitsDirectly(action_space.n)
        self._build(lr_schedule)

    def _build_mlp_extractor(self) -> None:
        default_lr = 0.01
        num_users = train_env.get_wrapper_attr('num_users')
        num_items = train_env.get_wrapper_attr('num_items')
        self.mlp_extractor = Net(self.observation_space, num_users, num_items, learning_rate=default_lr)

class ExtractPass(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space) -> None:
        super().__init__(observation_space, get_flattened_obs_dim(observation_space))
        self.flatten = nn.Flatten()

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        observations["user_id"] = observations["user_id"].int()
        return observations


In [5]:
#args = parse_args()
#args.llm_model='llama3.2'
#llm = load_LLM(args.llm_model)


In [6]:

if __name__ == "__main__":
    args = parse_args()
    #args.llm_model='llama3.2'
    llm = load_LLM(args.llm_model)

    dir_name = f"{args.llm_model}___{args.llm_rater}___{args.items_retrieval}___{args.user_dataset}___{args.news_dataset}___{args.perturbator}___{args.reward_shaping}___{args.seed}___{args.model_device}___{args.gamma}___{args.embedding_dim}___{args.learning_rate}"
    sanitized_dir_name = dir_name.replace('/', '_').replace(':', '_').replace('.', '_')
    save_path = f"/app/SUBERX/models/{sanitized_dir_name}"
    wandb_path = f"/app/wandb"
    os.makedirs(save_path, exist_ok=True)

    train_env = get_enviroment_from_args(llm, args)
    test_env = get_enviroment_from_args(llm, args, seed=args.seed + 600)

    policy_kwargs = dict(features_extractor_class=ExtractPass)
    train_env = StableBaselineWrapperNum(train_env)
    test_env = Monitor(StableBaselineWrapperNum(test_env))
    print("Checking training environment")
    check_env(train_env, warn=True)

    print("Checking Test Environment")
    check_env(test_env)

    print("Creating A2C Model")
    model = A2C(
        CustomActorCriticPolicy,
        train_env,
        verbose=1,
        policy_kwargs=policy_kwargs,
        device=args.model_device,
        learning_rate=linear_schedule(args.learning_rate),
        tensorboard_log=save_path,
        gamma=args.gamma,
        ent_coef=0.001,
    )
    print("Callback being created")
    combined_callback = CombinedCallback(save_freq=2500, log_freq=500, save_path=save_path, name_prefix="rl_model", verbose=1)
    callback = CallbackList([combined_callback])

    logger.info("Model starts learning")
    print("Model begining to learn")
    model.learn(total_timesteps=10000, progress_bar=True, callback=callback, tb_log_name="t_logs")

    logger.info("Model Ends Learning")
    print("Model Ends Learning")
    logger.info("Evaluating the Policy")
    mean_reward, std_reward = evaluate_policy(model, test_env, n_eval_episodes=50)
    logger.info(f"Mean reward: {mean_reward} +/- {std_reward}")

    reward_file_path = os.path.join(save_path, f"reward_{mean_reward:.2f}.txt")
    with open(reward_file_path, 'w') as file:
        file.write(f"Mean reward: {mean_reward} +/- {std_reward}\n")

    print(f"Reward information saved to {reward_file_path}")

    print(f"Mean Reward: {mean_reward} +/- {std_reward}")
    logger.info(f"Mean Reward: {mean_reward} +/- {std_reward}")


name is TheBloke/Llama-2-7b-Chat-GPTQ


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Notice:   0%|          | 0.00/112 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

USE_POLICY.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

...

Saving model checkpoint to 
/app/SUBERX/models/TheBloke_Llama-2-7b-Chat-GPTQ___2Shot_system_our___last_3___mind___small_mind_dataset___none___e
xp_decay_time___42___cuda_0___0_975___32___0_01/rl_model_7500_steps

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 42.9     |
|    ep_rew_mean        | 364      |
| time/                 |          |
|    fps                | 2        |
|    iterations         | 1500     |
|    time_elapsed       | 2649     |
|    total_timesteps    | 7500     |
| train/                |          |
|    entropy_loss       | -3.63    |
|    explained_variance | 0.308    |
|    learning_rate      | 0.0025   |
|    n_updates          | 1499     |
|    policy_loss        | 6.29     |
|    value_loss         | 4.94     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 45.5     |
|    ep_rew_mean        | 385      |
| time/                 |          |
|    fps                | 2        |
|    iterations         | 1600     |
|    time_elapsed       | 2819     |
|    total_timesteps    | 8000     |
| train/                |          |
|

Saving model checkpoint to 
/app/SUBERX/models/TheBloke_Llama-2-7b-Chat-GPTQ___2Shot_system_our___last_3___mind___small_mind_dataset___none___e
xp_decay_time___42___cuda_0___0_975___32___0_01/rl_model_10000_steps

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 44.3     |
|    ep_rew_mean        | 382      |
| time/                 |          |
|    fps                | 2        |
|    iterations         | 2000     |
|    time_elapsed       | 3564     |
|    total_timesteps    | 10000    |
| train/                |          |
|    entropy_loss       | -0.234   |
|    explained_variance | -0.953   |
|    learning_rate      | 5e-06    |
|    n_updates          | 1999     |
|    policy_loss        | -17      |
|    value_loss         | 2.83e+04 |
------------------------------------


Model Ends Learning
Reward information saved to /app/SUBERX/models/TheBloke_Llama-2-7b-Chat-GPTQ___2Shot_system_our___last_3___mind___small_mind_dataset___none___exp_decay_time___42___cuda_0___0_975___32___0_01/reward_287.42.txt
Mean Reward: 287.42 +/- 223.57442519214936
